In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [3]:
import os
import zipfile
import requests

def download_and_unzip(url, save_path):
    print("Downloading and extracting dataset...", end="")
    response = requests.get(url)
    with open(save_path, "wb") as f:
        f.write(response.content)

    try:
        with zipfile.ZipFile(save_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.dirname(save_path))
        print("✅ Done!")
    except zipfile.BadZipFile:
        print("❌ Failed to unzip file. The file may be corrupted.")

# Dataset download URL and paths
URL = "https://www.dropbox.com/scl/fi/vyvr7jbdvu8o174mbqgde/scut_data.zip?rlkey=fs8axkpxunwu6if9a2su71kxs&dl=1"
dataset_dir = "work"
dataset_zip_path = os.path.join(dataset_dir, "scut_data.zip")

# Make sure work/ directory exists
os.makedirs(dataset_dir, exist_ok=True)

# Download and unzip
download_and_unzip(URL, dataset_zip_path)


In [4]:
import torch
from transformers import VisionEncoderDecoderModel, TrOCRProcessor

model_name = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


2025-09-24 15:17:39.656350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758727060.028260      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758727060.132836      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [5]:
import os
from PIL import Image

dataset_path = "/kaggle/working/work/scut_data"

def load_data(img_folder, label_file):
    imgs, texts = [], []

    label_file_path = os.path.join(dataset_path, label_file)
    img_dir = os.path.join(dataset_path, img_folder)

    with open(label_file_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) >= 2:
                img_name, text = parts[0], "\t".join(parts[1:])
                imgs.append(os.path.join(img_dir, img_name))
                texts.append(text)

    return imgs, texts

# Load training and validation sets
train_imgs, train_texts = load_data("scut_train", "scut_train.txt")
val_imgs, val_texts = load_data("scut_test", "scut_test.txt")

print(f"Train Samples: {len(train_imgs)}, Val Samples: {len(val_imgs)}")

# Optional: show first 5 samples
for i in range(5):
    print(train_imgs[i], "->", train_texts[i])


Train Samples: 6052, Val Samples: 1651
/kaggle/working/work/scut_data/scut_train/000000.jpg -> BARNSTABLE
/kaggle/working/work/scut_data/scut_train/000001.jpg -> SHERIFF'S OFFICE
/kaggle/working/work/scut_data/scut_train/000002.jpg -> COUNTY
/kaggle/working/work/scut_data/scut_train/000003.jpg -> Levelezo-Lap.
/kaggle/working/work/scut_data/scut_train/000004.jpg -> Brefkort


In [6]:
# ================================================
# 2. Model: Load TrOCR Processor and Model
# ================================================

from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

# Use the smaller pre-trained TrOCR model
model_name = "microsoft/trocr-small-printed"

# Load processor (image + tokenizer) and model
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# Model configuration
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print("✅ TrOCR-small model loaded and ready on device:", device)


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ TrOCR-small model loaded and ready on device: cuda


In [7]:
import editdistance

def compute_cer(preds, targets):
    """
    Computes Character Error Rate (CER) between predicted and target texts.

    Args:
        preds (List[str]): List of predicted strings.
        targets (List[str]): List of ground truth strings.

    Returns:
        float: Average CER over all samples.
    """
    assert len(preds) == len(targets), "Predictions and targets must be the same length."

    total_edits = 0
    total_chars = 0

    for pred, target in zip(preds, targets):
        pred = pred.strip().replace(" ", "")
        target = target.strip().replace(" ", "")
        total_edits += editdistance.eval(pred, target)
        total_chars += len(target)

    if total_chars == 0:
        return 0.0

    return total_edits / total_chars

In [8]:
from transformers import Seq2SeqTrainingArguments

# Configuration class for training
class TrainingConfig:
    BATCH_SIZE = 2
    EPOCHS = 10
    LEARNING_RATE = 5e-5
    WEIGHT_DECAY = 0.01
    WARMUP_STEPS = 500

# HuggingFace training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr_finetuned",
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=TrainingConfig.BATCH_SIZE,
    per_device_eval_batch_size=TrainingConfig.BATCH_SIZE,
    num_train_epochs=TrainingConfig.EPOCHS,
    predict_with_generate=True,
    fp16=True,
    logging_strategy="epoch",
    save_total_limit=2,
    report_to="tensorboard",
    lr_scheduler_type="constant",
    dataloader_num_workers=4,
)


In [9]:
import os
from PIL import Image
from torch.utils.data import Dataset

class CustomOCRDataset(Dataset):
    """
    Custom Dataset class for OCR using TrOCR.

    Args:
        root_dir (str): Directory containing image files.
        label_file (str): Path to the label file (image_name <space/tab> label).
        processor (TrOCRProcessor): Hugging Face processor (tokenizer + image processor).
        max_target_length (int): Maximum length of target text sequence.
        augmentations (callable, optional): Image augmentations (e.g., Albumentations or torchvision).
    """

    def __init__(self, root_dir, label_file, processor, max_target_length=128, augmentations=None):
        self.root_dir = root_dir
        self.label_file = label_file
        self.processor = processor
        self.max_target_length = max_target_length
        self.augmentations = augmentations

        self.samples = []

        # Read label mappings
        with open(label_file, 'r', encoding='utf-8') as file:
            for line in file:
                line = line.strip()

                # Skip empty lines
                if not line:
                    continue

                # Handle space or tab as separator
                if '\t' in line:
                    file_name, label = line.split('\t', 1)
                elif ' ' in line:
                    file_name, label = line.split(' ', 1)
                else:
                    continue  # skip badly formatted lines

                # Normalize filename to 6-digit format if needed
                if file_name.endswith('.jpg') and len(file_name.split('.')[0]) < 6:
                    file_name = file_name.zfill(10)

                self.samples.append((file_name, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_name, label = self.samples[idx]
        img_path = os.path.join(self.root_dir, file_name)

        # Load image
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            raise FileNotFoundError(f"Image not found at {img_path}")

        # Apply augmentations if provided
        if self.augmentations:
            image = self.augmentations(image)

        # Use processor to prepare inputs and labels
        encoding = self.processor(
            image,
            text=label,
            padding="max_length",
            max_length=self.max_target_length,
            return_tensors="pt"
        )

        # Remove batch dimension (1, ...) → (...)
        item = {key: val.squeeze(0) for key, val in encoding.items()}

        return item


In [10]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os

# Load all image-text pairs from label file
with open("work/scut_data/scut_train.txt", "r", encoding="utf-8") as f:
    all_lines = [line.strip() for line in f if len(line.strip().split()) >= 2]

# Split into train and validation
train_lines, val_lines = train_test_split(all_lines, test_size=0.1, random_state=42)

# Save to temporary split files
train_label_file = "work/scut_data/train_split.txt"
val_label_file = "work/scut_data/val_split.txt"

with open(train_label_file, "w", encoding="utf-8") as f:
    f.write("\n".join(train_lines))

with open(val_label_file, "w", encoding="utf-8") as f:
    f.write("\n".join(val_lines))

# Define image root directory
image_root = "work/scut_data/scut_train"

# Now instantiate the datasets
train_dataset = CustomOCRDataset(
    root_dir=image_root,
    label_file=train_label_file,
    processor=processor,
    max_target_length=64
)

val_dataset = CustomOCRDataset(
    root_dir=image_root,
    label_file=val_label_file,
    processor=processor,
    max_target_length=64
)


In [11]:
from dataclasses import dataclass
from typing import Any, Dict, List
import torch

@dataclass
class DataCollatorForOCR:
    processor: Any
    padding: bool = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Get pixel values and already-tokenized label IDs
        pixel_values = [f["pixel_values"] for f in features]
        input_ids = [f["labels"] for f in features]  # These are already token IDs!

        # DO NOT call tokenizer() again — just pad
        batch = self.processor.tokenizer.pad(
            {"input_ids": input_ids},
            padding=self.padding,
            return_tensors="pt"
        )

        # Set padding tokens to -100 so they are ignored in loss
        labels = batch["input_ids"]
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        # Stack pixel values
        pixel_values = torch.stack(pixel_values)

        return {
            "pixel_values": pixel_values,
            "labels": labels
        }


In [12]:
from transformers import Seq2SeqTrainer


data_collator = DataCollatorForOCR(processor=processor)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.tokenizer,  # still needed for generation & decoding
    data_collator=data_collator    # custom collator here
)


# Start training
trainer.train()

# Save model & processor after training
model.save_pretrained("work/trocr_finetuned_model")
processor.save_pretrained("work/trocr_finetuned_model")


/tmp/ipykernel_36/172896966.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `

Step,Training Loss
1362,2.387700
2724,1.936900
4086,1.760200
5448,1.664000
6810,1.569100
8172,1.465900
9534,1.371900
10896,1.302300
12258,1.215300
13620,1.154200


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method

[]

In [13]:
import torch
from torch.utils.data import DataLoader
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
import editdistance

# === Load Fine-Tuned Model and Processor ===
model_path = "work/trocr_finetuned_model"

model = VisionEncoderDecoderModel.from_pretrained(model_path)
processor = TrOCRProcessor.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# === CER Metric Function ===
def compute_cer(preds, targets):
    """
    Computes Character Error Rate (CER) between predicted and target texts.
    """
    assert len(preds) == len(targets), "Predictions and targets must be the same length."

    total_edits = 0
    total_chars = 0

    for pred, target in zip(preds, targets):
        pred = pred.strip().replace(" ", "")
        target = target.strip().replace(" ", "")
        total_edits += editdistance.eval(pred, target)
        total_chars += len(target)

    if total_chars == 0:
        return 0.0

    return total_edits / total_chars

# === CER Scoring Function ===
def evaluate_model_cer(preds, targets):
    """
    Computes CER and assigns a score based on the project rubric.
    """
    cer_score = compute_cer(preds, targets)
    cer_percent = cer_score * 100

    if cer_percent <= 43:
        score = 50
    elif cer_percent <= 45:
        score = 40
    elif cer_percent <= 47:
        score = 30
    else:
        score = 0

    print(f"🔍 CER: {cer_percent:.2f}%")
    print(f"📊 Score Based on CER: {score} / 50")

    return cer_score, score

# === Inference and Evaluation ===
# Assuming val_dataset is already defined (see training section)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

predictions = []
ground_truths = []

for batch in val_dataloader:
    pixel_values = batch["pixel_values"].to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            pixel_values,
            max_length=64,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
        )

    # Decode predictions and ground truths
    pred_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    true_text = processor.batch_decode(batch["labels"], skip_special_tokens=True)[0]

    predictions.append(pred_text)
    ground_truths.append(true_text)

# === Compute CER and Score ===
cer, cer_score = evaluate_model_cer(predictions, ground_truths)


🔍 CER: 70.39%
📊 Score Based on CER: 0 / 50
